In [ ]:
!pip install python-pachyderm
!pip install boto3

In [ ]:
commit_id = "be82b14369c74ff7bbdf9866d6f701ee"

In [ ]:
import python_pachyderm as pc
client = pc.Client(host="pachd.pachyderm.svc.cluster.local")


In [ ]:
import pandas as pd
wines = client.get_file(("wines", "master", commit_id), "wine.csv")
wines_df = pd.read_csv(wines, delimiter=';')


In [ ]:
# Split data into training and test

wine_train = wines_df.sample(frac=0.7)
print(wine_train.shape)

In [ ]:
wine_test = wines_df.loc[~wines_df.set_index(list(wines_df.columns)).index.isin(wine_train.set_index(list(wine_train.columns)).index)]
print(wine_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
features = ['volatile acidity', 'residual sugar', 'chlorides', 'total sulfur dioxide', 'sulphates', 'alcohol']
wine_ln_reg = LinearRegression()
wine_ln_reg = wine_ln_reg.fit(wine_train[features], wine_train['quality'])


In [ ]:
import boto3, pickle, os

# Save the model to a file
with open('model.joblib', 'wb') as f:
    pickle.dump(wine_ln_reg, f)
